# Monod tutorial for Google Colab

This notebook runs the model, exports tabulated files, and generates plot images in the `results` folder.


## 1) Setup


In [ ]:
!pip install -q pybind11 numpy matplotlib pandas
!git clone https://github.com/michelanthony/TestAICoding.git
%cd TestAICoding
!pip install -e .


Repository URL used in this notebook: `https://github.com/michelanthony/TestAICoding.git`.


## 2) Run simulation and create tabulated files


In [ ]:
import os
import pandas as pd
import monod_bioreaction as m

os.makedirs("results", exist_ok=True)

X0, S0, t_max, dt = 0.1, 1.0, 10.0, 0.1
t_py, x_py, s_py = m.integrate(X0, S0, t_max, dt)

if m.monod_cpp is not None:
    t_cpp, x_cpp, s_cpp = m.integrate_with_pybind11(X0, S0, t_max, dt)
else:
    t_cpp, x_cpp, s_cpp = t_py, x_py, s_py
    print("monod_cpp not available, using Python values for C++ columns.")

df_compare = pd.DataFrame({
    "step": range(len(t_py)),
    "time": t_py,
    "x_python": x_py,
    "x_cpp": x_cpp,
    "s_python": s_py,
    "s_cpp": s_cpp,
})

df_compare.to_csv("results/monod_notebook_comparison.tsv", sep="\t", index=False)
print("Generated table file: results/monod_notebook_comparison.tsv")
df_compare.head(10)


## 3) Generate images with Matplotlib and save them to results


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].plot(t_py, x_py, "o-", label="Biomass Python")
axes[0].plot(t_cpp, x_cpp, "o-", label="Biomass C++", alpha=0.8)
axes[0].set_xlabel("Time (h)")
axes[0].set_ylabel("Concentration (g/L)")
axes[0].set_title("Biomass comparison")
axes[0].grid(True, alpha=0.3)
axes[0].legend()

axes[1].plot(t_py, s_py, "o-", label="Substrate Python")
axes[1].plot(t_cpp, s_cpp, "o-", label="Substrate C++", alpha=0.8)
axes[1].set_xlabel("Time (h)")
axes[1].set_ylabel("Concentration (g/L)")
axes[1].set_title("Substrate comparison")
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.savefig("results/monod_notebook_plot.png", dpi=150)
print("Generated image file: results/monod_notebook_plot.png")
plt.show()
